In [1]:
import uproot
import awkward as ak
import numpy as np
import math
import hist
import matplotlib.pyplot as plt
import os
import subprocess
import vector
import gc

In [2]:
vector.register_awkward() 

In [3]:
DATATYPE="mc"
assert((DATATYPE=="mc") or (DATATYPE=="data"))
BASEDIR="/pbs/throng/training/nantes-m2-rps-exp/data" # basedir where to look for runXXX.DATATYPE.root files
IS_MC=True if DATATYPE=="mc" else False

def data_file_path(run,is_mc=IS_MC,dest=BASEDIR):
    datatype="mc" if is_mc else "data"
    print({dest},"/run",{run},".",{datatype},".root")
    return f"{dest}/run{run}.{datatype}.root"

In [4]:
SAMPLE_RUNS=[291694,291399]

In [5]:
file = uproot.open(data_file_path(SAMPLE_RUNS[0],IS_MC))
events = file["eventsTree"]
eventsGen = file["genTree"]
events.show()
eventsGen.show()

{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {291694} . {'mc'} .root
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
runNumber            | int32_t                  | AsDtype('>i4')
xVtx                 | double                   | AsDtype('>f8')
yVtx                 | double                   | AsDtype('>f8')
zVtx                 | double                   | AsDtype('>f8')
isCINT               | bool                     | AsDtype('bool')
isCMSL               | bool                     | AsDtype('bool')
isCMSH               | bool                     | AsDtype('bool')
isCMLL               | bool                     | AsDtype('bool')
isCMUL               | bool                     | AsDtype('bool')
nMuons               | int32_t                  | AsDtype('>i4')
Muon_E               | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_Px              | st

We will just print here the number of entries (events) in the file for the record

In [6]:
print(events.num_entries)
print(eventsGen.num_entries)

40000
40000


In [7]:
def getTracks(events):
    return ak.zip({"px":events["Muon_Px"],
                    "py":events["Muon_Py"],
                    "pz":events["Muon_Pz"],
                    "E":events["Muon_E"],
                    "charge":events["Muon_Charge"],
                    "thetaAbs":events["Muon_thetaAbs"],
                    "matched":events["Muon_matchedTrgThreshold"],
                    "n":events["nMuons"]},
                    with_name='Momentum4D')

def getTracksGen(events):
    return ak.zip({"px":events["Muon_GenPx"],
                    "py":events["Muon_GenPy"],
                    "pz":events["Muon_GenPz"],
                    "E":events["Muon_GenE"],
                    "n":events["nMuonsGen"],
                    "label":events["Muon_GenLabel"],
                    "mother":events["Muon_GenMotherPDGCode"]},
                    with_name='Momentum4D')

Now we will design a function to run over the data with uproot. 
In the iterate, we define all the varaible we want to access for each events

We first select good events and track using lambdas in python. What is a good event or a good track will be define later when we call the function "scan"

In [8]:
#print(dir(vector.backends.awkward.MomentumArray4D))

In [9]:
import pandas
l1 = pandas.read_csv('../data/counters.offline.csv')['run']
l2 = [int(e) for e in os.popen("ls /pbs/throng/training/nantes-m2-rps-exp/data/ | grep 'mc' | cut -c 4-9").read().split('\n')[0:-2]]
#list(set(l1) & set(l2))

In [10]:
#def invariant_mass(part1,part2):
#    ''' Not used anymore '''
#    px = part1.px+part2.px
#    py = part1.py+part2.py
#    pz = part1.pz+part2.pz 
#    E1 = part1.e
#    E2 = part2.e
#    return np.sqrt((E1+E2)**2-(px**2+py**2+pz**2))

def Momentum4D(events):
    return ak.zip({
        "px": events["0"].px + events["1"].px,
        "py": events["0"].py + events["1"].py,
        "pz": events["0"].pz + events["1"].pz,
        "E" : events["0"].E  + events["1"].E},
        with_name="Momentum4D")

In [11]:
def makePairsMomentum(tracks):
    # Combinations & Keep opposite charges only 
    C = ak.combinations(tracks,2)
    C_OS = C[(C["0"].charge+C["1"].charge)==0]
    P = Momentum4D(C_OS)
    return P
    
def scan(dataDescription,
            fdata, fJPsi, fPsi2S,
            eventSelector=lambda x:[True]*len(x),
            trackSelector=lambda x:[True]*len(x),
            pairSelector=lambda x:[True]*len(x),
            verbose:bool=False):
    """ Loop over data to write masses to files.
        
        :param: dataDescription: is anything uproot.iterate can take.
                typical something like run*.data.root:eventsTree in our case
        :param: eventSelector: returns an array of bool from an array of events
        :param: trackSelector: returns an array of bool from an array of tracks
    """
    #MCEntries = ["Muon_MCLabel", "Muon_MCPDGCode"]
    eventsTreeEntries = ["isCINT","isCMUL","isCMSL","nMuons","Muon_Px","Muon_Py","Muon_Pz","Muon_E","Muon_Charge","Muon_thetaAbs","Muon_matchedTrgThreshold"]
    genTreeEntries = ["nMuonsGen","Muon_GenPx","Muon_GenPy","Muon_GenPz","Muon_GenE","Muon_GenLabel","Muon_GenMotherPDGCode"]
    if 1:
        for batch in uproot.iterate(dataDescription+".data.root:eventsTree", eventsTreeEntries, report=True):
            events = batch[0]
            # Event selection
            eventSelection = eventSelector(events)
            tracks = getTracks(events[eventSelection])          
            # Track selection
            trackSelection = trackSelector(tracks)
            tracks = tracks[trackSelection]
            # Combinations
            P = makePairsMomentum(tracks)
            pairSelection = pairSelector(P)
            # Save all masses
            np.save(fdata, ak.flatten(P[pairSelection].mass).to_numpy()) 
            
            if verbose:
                print(batch[1])
            gc.collect()
        
        for batch, batchGen in zip(uproot.iterate(dataDescription+".mc.root:eventsTree", eventsTreeEntries, report=True),
                                   uproot.iterate(dataDescription+".mc.root:genTree", genTreeEntries, report=True)):
            events = batch[0]
            # Event selection
            eventSelection = eventSelector(events)
            tracks = getTracks(events[eventSelection])          
            # Track selection
            trackSelection = trackSelector(tracks)
            tracks = tracks[trackSelection]
            # Apply selections on gen
            eventsGen = batchGen[0]

            tracksGen = getTracksGen(eventsGen[eventSelection])
            try:  # Temporary fix, but it's OK, it's just for MC initial fit
                tracksGen = tracksGen[trackSelection]  # fail for some runs (e.g 290374) bc of differents track shapes        
                # J/psi and psi(2S) selection
                for code, f in zip((443, 100443), (fJPsi, fPsi2S)):
                    motherSelection = tracksGen.mother==code
                    tracksReso = tracks[motherSelection]
                    # Combinations
                    P = makePairsMomentum(tracksReso)
                    pairSelection = pairSelector(P)
                    # Save all masses
                    np.save(f, ak.flatten(P[pairSelection].mass).to_numpy()) 
            except Exception as e:
                np.save(fJPsi, []) 
                np.save(fPsi2S, []) 
                pass        
            

In [12]:
%%time
runid = "{290323,290327,290848,291361,291360,291362,290853,290860,291373,290374,290375,291399,291400,290894,290895,290404,291943,291944,291948,291953,290932,290423,291447,290935,290425,290427,291451,291453,291976,291982,290456,290458,290459,291482,291485,290975,290980,290469,292012,291002,291003,291004,291005,290501,292040,292060,292061,292062,291041,290539,290540,292075,292077,292080,290549,290553,291590,292106,292108,292109,292115,290590,291618,291622,291624,292140,290612,292160,292162,292163,292164,292166,290632,291657,292168,292192,290658,290660,291690,291692,291694,291698,291706,290687,290692,290696,290699,292242,292265,291755,292269,292270,291760,292273,292274,290742,291769,291263,290764,290766,291283,291284,291285,291795,291796,290776,291803,290787}"
#runid = "{290853,290327,290848}"
#runid = "290374"
os.system(f"ls {BASEDIR}/run{runid}.data.root | wc -l")
if 1:
    pTcuts = [0, 1, 2, 3, 4, 5, 6, 8]
    #pTcuts = [0, 1000]
    for i in range(len(pTcuts)-1):
        pTl = pTcuts[i]
        pTu = pTcuts[i+1]
        # Create file names
        suffix = f"pT{pTl}-{pTu}.npy"
        fiddata = "mass/data."+suffix
        fidJPsi = "mass/JPsi."+suffix
        fidPsi2S = "mass/Psi2S."+suffix
        # Clear files data
        for f in (fiddata, fidJPsi, fidPsi2S):
            os.system("> "+f)
        # Open files for writing
        fdata = open(fiddata,"ab")
        fJPsi = open(fidJPsi,"ab")
        fPsi2S = open(fidPsi2S,"ab")
        cuttrack = lambda x: (x.thetaAbs < 10) & (x.thetaAbs > 2) & (x.pt > 0.5) & (x.eta < -2.5) & (x.eta > -4)
        cutpair = lambda x: (pTl<x.pt)&(x.pt<pTu)
        scan(dataDescription=f"{BASEDIR}/run{runid}",
                fdata=fdata, fJPsi=fJPsi, fPsi2S=fPsi2S,
                eventSelector=lambda x: x["isCMUL"]==True,
                trackSelector=cuttrack,
                pairSelector=cutpair,
                verbose=True
            )
        for f in (fdata, fJPsi, fPsi2S):
            f.close()


108
<Report start=0 stop=2139140 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290323.data.root:/eventsTree;1'>
<Report start=2139140 stop=2329535 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290327.data.root:/eventsTree;1'>
<Report start=2329535 stop=4902978 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290848.data.root:/eventsTree;1'>
<Report start=4902978 stop=5125248 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290848.data.root:/eventsTree;1'>
<Report start=5125248 stop=5682939 source='/pbs/throng/training/nantes-m2-rps-exp/data/run291361.data.root:/eventsTree;1'>
<Report start=5682939 stop=8151330 source='/pbs/throng/training/nantes-m2-rps-exp/data/run291360.data.root:/eventsTree;1'>
<Report start=8151330 stop=8740215 source='/pbs/throng/training/nantes-m2-rps-exp/data/run291360.data.root:/eventsTree;1'>
<Report start=8740215 stop=10927537 source='/pbs/throng/training/nantes-m2-rps-exp/data/run291362.data.root:/eventsTree;1'>
<Report start=109